# NLP 3 과제
> 인공지능 스터디 여덟 번째 과제에 오신 것을 환영합니다! 강의를 들으면서 배운 다양한 지식들을 실습을 통해서 활용해 볼 시간을 가질 것입니다!

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> ROUGE-L
```python
다음 두개의 모델이 예측한 문장에 대해서 ROUGE-L Score를 실제로 구해봅시다.

score 이름인 'Recall-Oriented Understudy for Gisting Evaluation' 에 Recall이 있듯이
recall 값을 주로 metric에 사용하므로 recall만을 계산해봅시다.

정답 문장: "always be with you"
예측 문장(from Model 1): "always be without you"
예측 문장(from Model 2): "you always be with me"
```

```python
😉
#TODO
Model 1 score : 3/4
Model 2 score : 3/4
```

# 👨‍💻 <font color='green'><b>[ 실습 ]</b></font> Huggingface 훑어보기
```python
Hugging Face 는 자연어 처리 분야에서 흔히 사용되는 오픈소스 플랫폼입니다.
제공하는 Huggingface transformers library 사용법에 대해서 살펴봅시다.
```
* 아래의 링크에서 더 자세하고 꼼꼼한 튜토리얼을 공부할 수 있습니다. 참고해보세요 🤗

    https://huggingface.co/docs/transformers/notebooks


* 아래의 링크에서 huggingface에서 제공하는 라이브러리에 대한 discussion을 살펴볼 수 있습니다.

  https://discuss.huggingface.co/

### 🔧  초기 설정
```python
transformers 라이브러리를 설치해봐요.
```

In [10]:
! pip install transformers -q

## Transformers 라이브러리 소개
```python
NLP 모델은 아래와 같은 Pipeline으로 구성됩니다

'Input' -> 'Tokenization' -> 'Model training/Inference' -> 'Post-Processing'(task dependent) -> 'Output'

이러한 pipeline을 모두 scratch부터 구현해보는것은 매우 의미있겠지만,

task에 알맞는 모델을 빠르게 구현하기 위해서는 반복적인 작업을 규칙에 따라 정리 할 필요성이 있습니다.

🤗 'Huggingface transformers' 🤗 는 모델을 쉽고 빠르게 생성하고, 학습하고, 배포하기 위해 만들어진 Highlevel Library 입니다.

즉, 우리는 Huggingface를 사용함으로서 반복되던 작업을 간편하게 수행할 수 있고, '모델 개발에 초점을 맞추어 개발 및 연구'를 진행할 수 있게됩니다.
```

### Tokenizer 살펴보기
```python
텍스트 데이터를 모델이 알아들을 수 있는 형태로 변환하기 위해서는 'tokenization' 과정을 거쳐 encoding을 진행해야합니다.

이 때 text data를 token화 하고 특정 숫자로 encoding 하는 과정을 모두 수행하는것이 transformers tokenizer 역할입니다.

과거에 PLM(Pretrained language model)이 없었던 시기에는 가지고 있는 데이터를 기반으로 parsing을 진행(성능 좋은 Parser를 사용)한 후 dictionary를 데이터별로 만들어 직접 숫자를 부여했습니다. 하지만 최근에는 PLM을 활용한 토크나이저를 사용합니다.
```

In [11]:
from transformers import AutoTokenizer

example = "나는 이제 너 없이도 너를 좋아할 수 있다."

# tokenization 결과를 보여줍니다.
model_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

```python
tokenization 결과를 확인 해봅시다.
bert-base-cased는 영어에 대한 tokenizer이므로 한글을 전혀 이해하지 못합니다. 따라서 tokenizer output은 '[unk]'(unknown token)으로 나오는 것을 확인할 수 있습니다.
```

In [12]:
print('tokenization 결과 : ', tokenizer.tokenize(example))
print('tokenization + encoding 결과 : ', tokenizer.encode(example))

tokenization 결과 :  ['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '.']
tokenization + encoding 결과 :  [101, 100, 100, 100, 100, 100, 100, 100, 100, 119, 102]


```python
모델을 klue/bert-base 로 변경하여 tokenization을 다시 해봅시다.
klue/bert-base는 한글에 대한 모델이므로 한글을 인식할 수 있습니다.

여기서 KLUE(Korean Language Understanding Evaluation)란, 한국어 모델의 성능을 평가하기 위한 데이터셋입니다.

언어에 알맞은 적절한 tokenizer를 사용해야 함을 알 수 있습니다.
```

In [13]:
example = "나는 이제 너 없이도 너를 좋아할 수 있다."

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [14]:
print('tokenization 결과 : ', tokenizer.tokenize(example))
print('tokenization + encoding 결과 : ', tokenizer.encode(example))

tokenization 결과 :  ['나', '##는', '이제', '너', '없이', '##도', '너', '##를', '좋아할', '수', '있', '##다', '.']
tokenization + encoding 결과 :  [2, 717, 2259, 3699, 743, 3806, 2119, 743, 2138, 26020, 1295, 1513, 2062, 18, 3]


#### tokenizer 사용시 주의사항을 확인해 봅시다.
```python
1. train data의 언어를 이해 할 수 있는 tokenizer인지 확인
2. 사용하고자 하는 pretrained model과 동일한 tokenizer인지 확인

```
---

### Config 불러오기
```python
사전 학습 모델을 사용하기 위해서는 사전학습 모델이 가진 setting(=Config)을 그대로 가져와야합니다.

모델마다 vocab size, hidden dimension등 각각의 파라미터 세팅이 상이하므로

transformers는 이 정보를 Config로 쉽게 불러올 수 있는 기능을 제공합니다.

'Auto' class로 모델 이름만 넣어준다면 편리하게 configuration을 가져올 수 있습니다.

```

In [15]:
from transformers import AutoConfig

model_name =  'klue/bert-base'

# pretrained 모델과 동일한 configuration을 가져옵니다.
model_config = AutoConfig.from_pretrained(model_name)

In [16]:
model_config

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

```python
어떤 경우에는 config를 수정하여 사용하기도 하는데, 바꾸어도 되는 config와 바꾸지 말아야 하는 config가 정해져 있습니다.

'바꾸면 안되는 config'
-> Pretrained model 사용시 hidden dim등 이미 정해져 있는 모델의 아키텍쳐 세팅은 수정하면 안됩니다.
이를 수정해버릴 경우 에러가 발생하거나, 잘못된 방향으로 학습 될 수 있습니다.

'바꾸어도 되는 config'
-> vocab의 경우 special token을 추가한다면 config를 추가한 vocab의 개수만큼 추가하여 학습해야합니다.

```
---

### Pretrain model 불러오기
```python
transformers의 가장 강력한 기능은 사전학습된 모델을 쉽게 불러오고, 사용할 수 있다는 것입니다.

단 세 줄로 원하는 pretrained model을 불러오고 사용할 수 있습니다.

우리는 해당모델을 그대로 사용할 수도 있고, 추가적으로 학습을 진행하여 내 데이터에 맞는 모델로 사용할수도 있습니다.

* '.from_config()' 는 config 그대로 모델을 가져오는 method 입니다. 즉 사전학습된 weight을 가져오는게 아니니 주의해야합니다.
* '.from_pretrained()' 는 model config에 해당하는 모델을 가져오고, '사전학습된 weight를 가져옵니다'. 스스로 학습한 모델을 불러오려면 model_name 부분에 model이 저장된 directory를 입력하면 됩니다.
```

```python
transformers는 두가지 타입의 모델을 제공하고 있습니다.

* 기본 모델
-> hidden state가 출력되는 기본 모델

* downstream task 모델
-> 일반적인 task를 쉽게 수행할 수 있도록 미리 기본 모델 + output head가 설정된 모델
-> output은 task에 적합한 dimension으로 미리 정의되어있음

아래 코드에서는 downstream task 모델, 즉 MRC를 위한 모델을 불러오고 있습니다.
```

In [17]:
from transformers import AutoConfig, AutoModelForQuestionAnswering
# Download configuration from huggingface.co and cache.

model_name = 'klue/bert-base'

# pretrained 모델과 동일한 configuration을 가져옵니다.
model_config = AutoConfig.from_pretrained(model_name)

# 모델을 정의합니다.

# option 1 : config에서 정의한 모델을 가져오기 (initial)
# model = AutoModelForQuestionAnswering.from_config(config)

# option 2 : config에서 정의한 사전학습된 모델을 가져오기 (pretrained)
model = AutoModelForQuestionAnswering.from_pretrained(
        model_name, config=model_config
    )

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


```python
이제 우리는 가장 기본적인 tokenizer, config, model를 loading하는 방법에 대해 배웠습니다.

아래의 코드처럼 세가지를 한번에 load하여 사용하면 됩니다.
```

In [18]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config,
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Trainer
```python
모델까지 불러왔다면 이제 학습을 진행해 봅시다.

반복되는 Training loop를 효과적으로 모듈화 시켜놓은것이 transformers의 'trainer'입니다.
  
덕분에 우리는 매 모델을 학습하기 위해 training loop를 구현하는 과정을 단 몇줄만에 해결할 수 있습니다.

아래와 같은 구조로 trainer를 사용할 수 있습니다.

1. TrainingArguments 설정 (학습을 위한 하이퍼파라미터를 설정합니다.)
2. Trainer 호출
3. 학습 / 추론 진행
```

```python
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model_name = 'klue/bert-base'

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)


trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
```
---


### Datasets
```python
trainer까지 알게 되었지만 아직 Dataset이 준비되지 않았습니다.

Huggingface에서 제공하는 datasets 라이브러리를 통해 LG CNS가 제공한 한국어 MRC 데이터 셋인 KorQuAD 데이터 셋을 확인 해봅시다.
```
* KorQuAD 데이터셋 사이트입니다
    : https://korquad.github.io/KorQuad%201.0/

In [19]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


#### 데이터셋 불러오기

In [20]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

#### 데이터셋 둘러보기

In [22]:
dataset["train"][0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]}}

```python
랜덤으로 하나의 데이터를 확인해봅시다!
```

In [23]:
import random
pick = random.randint(0, len(dataset['train'])-1)
example = dataset['train'][pick]
example

{'id': '6570007-2-2',
 'title': '한국투자공사',
 'context': '심상정 민주노동당 의원은 "현재의 CIO 구안 옹은 1998년-2006년 푸르덴셜에서의 자산운용 경력이 전부인데, 그나마 국제투자사업부문의 경력은 1개월 밖에 안 된다. 더구나 GIC(싱가포르투자청)에 두 번이나 입사지원을 냈다 탈락했다는 주장이 나오고 있다"며 "직원 가운데 분야별 투자전문직원은 1~2명 정도이고 운영위원 중에도 국제금융시장 실무경험자가 거의 없는 등 도저히 국민의 세금을 국제금융시장에 투자해 이윤을 남길 경험과 능력이 없다"고 한국투자공사의 전문성 부족을 질타했다. 또 "이강원 한국투자공사 초대 사장의 한국외환은행장 당시의 비서실장을 상무로, 리스크관리부 차장이 한국투자공사 리스크관리팀 부장으로, 이 사장의 친인척을 경영기획팀 차장으로 앉히는 등 개인인맥을 심어 조직을 주물러왔다는 의혹도 무성하다"며 "또한 일부 운영위원의 경우 미국에서 열린 운영위원회 참석 비용이 하루 1천만 원에 달하였고, 또 다른 운영위원은 경쟁사의 사외이사로 재직하는 등 도덕적 해이가 심각하다"고 질타했다. 마지막으로 “한국투자공사를 폐지하고 한국은행이 외화자산 운용능력을 보강해서 한국투자공사의 업무를 대신해야 한다”고 주장했다.',
 'question': '한국투자공사 초대 사장의 이름은?',
 'answers': {'text': ['이강원'], 'answer_start': [277]}}

```python
'answer_start'는 context에서 answer가 시작하는 부분을 가리킵니다.
```

In [24]:
context = example['context']
answer_text = example['answers']['text'][0]
answer_start = example['answers']['answer_start'][0]
answer_end = answer_start + len(answer_text)
print('Answer from context', context[answer_start:answer_end])

Answer from context 이강원


#### Tokenizing 해보기

In [25]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

```python
참고.

모델 뒤에 붙은 cased 에 의미에 대해 알아봅시다.

* cased vs. uncased

1. lowercasing
BERT uncased : OpenGenus -> opengenus
BERT cased : OpenGenus

2. accent striping
BERT uncased : OpènGènus -> opengenus
BERT cased : OpènGènus

3. unicode normalization (NFD)
BERT uncased : 안녕 -> ㅇ, ㅏ, ㄴ, ㄴ, ㅕ, ㅇ
BERT cased : 안녕

```

In [26]:
example_text = dataset["train"][0]['question']
example_text

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'

```python
토크나이징 결과를 확인해 봅시다.
```

In [27]:
tokenizer.tokenize(example_text)

['바',
 '##그',
 '##너',
 '##는',
 '괴',
 '##테',
 '##의',
 '파',
 '##우스',
 '##트를',
 '읽',
 '##고',
 '무',
 '##엇',
 '##을',
 '쓰',
 '##고',
 '##자',
 '했',
 '##는',
 '##가',
 '?']

```python
토크나이징된 단어의 vocab에서 위치를 확인해 봅시다.
```

In [28]:
tokenizer(example_text)['input_ids']

[101,
 9318,
 78136,
 70162,
 11018,
 8905,
 119351,
 10459,
 9901,
 89108,
 101825,
 9642,
 11664,
 9294,
 119137,
 10622,
 9511,
 11664,
 13764,
 9965,
 11018,
 11287,
 136,
 102]

```python
토크나이징된 문장을 다시 원 문장으로 복구할 수도 있습니다.
```

In [29]:
' '.join(tokenizer.tokenize(example_text)).replace(' ##', '')

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가 ?'

In [30]:
tokenizer.decode(tokenizer(example_text)['input_ids'])

'[CLS] 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가? [SEP]'

```python
decode를 통해 복구하였을 때에는 bert에서 사용되는 token이 추가된 것을 확인할 수 있습니다.
```

```python
Hugging Face에서 제공하는 다양한 모델과 데이터셋은 여러분이 간편하게 자연어 처리 모델을 학습하고 활용할 수 있게 해줍니다.

또한, 생각한 아이디어를 실제 프로젝트에 적용할 때, 이 풍부한 리소스를 통해 빠르게 구현하고 실험할 수 있습니다.
```

### 🎉🎉🎉 8주차 과제 완료! 🎉🎉🎉
```python
🐙
마지막 과제까지 모두 수고하셨어요! 여러분의 성실한 노력과 열정에 감사합니다.

언제든지 도움이 필요하면 자유롭게 물어봐주세요. 수고하셨고 감사합니다!
```